Calculating the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data. This process involves data cleaning and calculating wins and losses.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns = {cities.columns[1]:'Population'})
print(cities.columns)
#print(cities["NHL"])
nhl_df = nhl_df[(nhl_df['year']==2018)]
nhl_df = nhl_df[(nhl_df['team'] != 'Atlantic Division') & (nhl_df['team'] != 'Metropolitan Division') & 
                (nhl_df['team'] != 'Central Division') & (nhl_df['team'] != 'Pacific Division')]
               
nhl_df.replace('\*', '',regex = True,inplace = True)

nhl_df[['city','team']] = nhl_df['team'].str.split(' ',1,expand = True)
nhl_df.at[1,'city']='Tampa Bay'
nhl_df.at[1,'team']='Lightning'
nhl_df.at[14,'city']='New Jersey'
nhl_df.at[14,'team']='Devils'
nhl_df.at[16,'city']='New York'
nhl_df.at[16,'team']='Islanders'
nhl_df.at[17,'city']='New York'
nhl_df.at[17,'team']='Rangers'
nhl_df.at[23,'city']='St. Louis'
nhl_df.at[23,'team']='Blues'
nhl_df.at[29,'city']='San Jose'
nhl_df.at[29,'team']='Sharks'
nhl_df.at[30,'city']='Los Angeles'
nhl_df.at[30,'team']='Kings'

cities_NHL = cities.iloc[:,[0,1,5]]
cities_NHL.replace({'NHL':r'\[.+\]'}, {'NHL':""},regex = True,inplace = True)

cities_NHL.at[0,'NHL'] = 'Rangers'
nr = pd.Series({cities_NHL.columns[0]:cities_NHL.iat[0,0],cities_NHL.columns[1]:cities_NHL.iat[0,1],'NHL':'Islanders'})
cities_NHL = cities_NHL.append(nr,ignore_index = True)
nr['NHL'] = 'Devils'
cities_NHL = cities_NHL.append(nr,ignore_index =True)

cities_NHL.at[1,'NHL'] = 'Kings'
nr = pd.Series({cities_NHL.columns[0]:cities_NHL.iat[1,0],cities_NHL.columns[1]:cities_NHL.iat[1,1],'NHL':'Ducks'})
cities_NHL = cities_NHL.append(nr,ignore_index = True)

new_df_nhl = cities_NHL.merge(nhl_df,left_on='NHL',right_on='team',how='right')
#print(list(nhl_df["team"].sort_values()))
#print(list(cities_NHL["NHL"].sort_values()))
new_df_nhl['W'] = pd.to_numeric(new_df_nhl['W'])
new_df_nhl['L'] = pd.to_numeric(new_df_nhl['L'])
new_df_nhl['Population'] = pd.to_numeric(new_df_nhl['Population'])

def corr_data(df):
    ans = df.groupby('Metropolitan area').agg({'W':sum,'L':sum,'Population':min})
    ans['ratio'] = ans['W']/(ans['W']+ans['L'])
    return ans
    
    
def nhl_correlation(): 
    
    ans = corr_data(new_df_nhl)
    population_by_region = ans[new_df_nhl.columns[1]] # pass in metropolitan area population from cities
    win_loss_by_region = ans['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

print(nhl_correlation())

Index(['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL'], dtype='object')
(0.012308996455744249, 0.9504308637909502)


Calculating the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns = {cities.columns[1]:'Population'})
nba_df = nba_df[(nba_df['year']==2018)]
nba_df.replace({'team':r'[\*\d\(\)]'}, '',regex = True,inplace = True)
nba_df['team'] = nba_df['team'].str.strip()
nba_df[['city','team']] = nba_df['team'].str.split(' ',1,expand = True)

nba_df.at[10,'city']='New York'
nba_df.at[10,'team']='Knicks'
nba_df.at[16,'city']='San Farncisco Bay Area'
nba_df.at[16,'team']='Warriors'
nba_df.at[18,'city']='Oklahoma City'
nba_df.at[18,'team']='Thunder'
nba_df.at[20,'city']='New Orleans'
nba_df.at[20,'team']='Pelicans'
nba_df.at[21,'city']='San Antonio'
nba_df.at[21,'team']='Spurs'
nba_df.at[24,'city']='Los Angeles'
nba_df.at[24,'team']='Clippers'
nba_df.at[25,'city']='Los Angeles'
nba_df.at[25,'team']='Lakers'
nba_df.at[2,'team']='76ers'

#print(nba_df)

cities_NBA = cities.iloc[:,[0,1,4]]
cities_NBA.replace({'NBA':r'\[.+\]'}, {'NBA':""},regex = True,inplace = True)

cities_NBA.at[0,'NBA'] = 'Knicks'
nr = pd.Series({cities_NBA.columns[0]:cities_NBA.iat[0,0],cities_NBA.columns[1]:cities_NBA.iat[0,1],'NBA':'Nets'})
cities_NBA = cities_NBA.append(nr,ignore_index = True)

cities_NBA.at[1,'NBA'] = 'Lakers'
nr = pd.Series({cities_NBA.columns[0]:cities_NBA.iat[1,0],cities_NBA.columns[1]:cities_NBA.iat[1,1],'NBA':'Clippers'})
cities_NBA = cities_NBA.append(nr,ignore_index = True)

new_df_nba = cities_NBA.merge(nba_df,left_on='NBA',right_on='team',how='right')

new_df_nba['W'] = pd.to_numeric(new_df_nba['W'])
new_df_nba['L'] = pd.to_numeric(new_df_nba['L'])
new_df_nba['Population'] = pd.to_numeric(new_df_nba['Population'])


def nba_correlation():
    ans = corr_data(new_df_nba)
    
    population_by_region = ans['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = ans['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)

print(nba_correlation())

(-0.17657160252844617, 0.36874741604463)


Calculating the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns = {cities.columns[1]:'Population'})
mlb_df = mlb_df[(mlb_df['year']==2018)]
mlb_df[['city','team']] = mlb_df['team'].str.split(' ',1,expand = True)

mlb_df.at[1,'city']='New York'
mlb_df.at[1,'team']='Yankees'
mlb_df.at[2,'city']='Tampa Bay'
mlb_df.at[2,'team']='Rays'
mlb_df.at[9,'city']='Kansas City'
mlb_df.at[9,'team']='Royals'
mlb_df.at[13,'city']='Los Angeles'
mlb_df.at[13,'team']='Angels'
mlb_df.at[18,'city']='New York'
mlb_df.at[18,'team']='Mets'
mlb_df.at[22,'city']='St.Louis'
mlb_df.at[22,'team']='Cardinals'
mlb_df.at[25,'city']='Los Angeles'
mlb_df.at[25,'team']='Dodgers'
mlb_df.at[28,'city']='San Francisco'
mlb_df.at[28,'team']='Giants'
mlb_df.at[29,'city']='San Diego'
mlb_df.at[29,'team']='Padres'


cities_mlb = cities.iloc[:,[0,1,3]]
cities_mlb.replace({'MLB':r'\[.+\]'}, {'MLB':""},regex = True,inplace = True)

cities_mlb.at[0,'MLB'] = 'Yankees'
nr = pd.Series({cities_mlb.columns[0]:cities_mlb.iat[0,0],cities_mlb.columns[1]:cities_mlb.iat[0,1],'MLB':'Mets'})
cities_mlb = cities_mlb.append(nr,ignore_index = True)

cities_mlb.at[1,'MLB'] = 'Dodgers'
nr = pd.Series({cities_mlb.columns[0]:cities_mlb.iat[1,0],cities_mlb.columns[1]:cities_mlb.iat[1,1],'MLB':'Angels'})
cities_mlb = cities_mlb.append(nr,ignore_index = True)

cities_mlb.at[2,'MLB'] = 'Giants'
nr = pd.Series({cities_mlb.columns[0]:cities_mlb.iat[2,0],cities_mlb.columns[1]:cities_mlb.iat[2,1],'MLB':'Athletics'})
cities_mlb = cities_mlb.append(nr,ignore_index = True)

cities_mlb.at[3,'MLB'] = 'Cubs'
nr = pd.Series({cities_mlb.columns[0]:cities_mlb.iat[3,0],cities_mlb.columns[1]:cities_mlb.iat[3,1],'MLB':'White Sox'})
cities_mlb = cities_mlb.append(nr,ignore_index = True)


new_df_mlb = cities_mlb.merge(mlb_df,left_on='MLB',right_on='team',how='right')
new_df_mlb['W'] = pd.to_numeric(new_df_mlb['W'])
new_df_mlb['L'] = pd.to_numeric(new_df_mlb['L'])
new_df_mlb['Population'] = pd.to_numeric(new_df_mlb['Population'])

def mlb_correlation(): 
    ans = corr_data(new_df_mlb)
    population_by_region = ans['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = ans['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

   
    return stats.pearsonr(population_by_region, win_loss_by_region)

print(mlb_correlation())



(0.1505230448710485, 0.4629669779770907)



Calculating the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns = {cities.columns[1]:'Population'})
nfl_df = nfl_df[(nfl_df['year']==2018)]
nfl_df.replace('[\*\+]', '',regex = True,inplace = True)
nfl_df = nfl_df[(nfl_df['team'] != 'NFC North') & (nfl_df['team'] != 'NFC South') & (nfl_df['team'] != 'NFC East') & (nfl_df['team'] != 'NFC West') & 
                (nfl_df['team'] != 'AFC North') & (nfl_df['team'] != 'AFC South') & (nfl_df['team'] != 'AFC East') & (nfl_df['team'] != 'AFC West')]            
nfl_df[['city','team']] = nfl_df['team'].str.split(' ',1,expand = True)
nfl_df.at[1,'city']='New England'
nfl_df.at[1,'team']='Patriots'
nfl_df.at[4,'city']='New York'
nfl_df.at[4,'team']='Jets'
nfl_df.at[16,'city']='Kansas City'
nfl_df.at[16,'team']='Chiefs'
nfl_df.at[17,'city']='Los Angeles'
nfl_df.at[17,'team']='Chargers'
nfl_df.at[24,'city']='New York'
nfl_df.at[24,'team']='Giants'
nfl_df.at[28,'city']='Green Bay'
nfl_df.at[28,'team']='Packers'
nfl_df.at[31,'city']='New Orleans'
nfl_df.at[31,'team']='Saints'
nfl_df.at[34,'city']='Tampa Bay'
nfl_df.at[34,'team']='Buccaneers'
nfl_df.at[36,'city']='Los Angeles'
nfl_df.at[36,'team']='Rams'
nfl_df.at[38,'city']='San Francisco'
nfl_df.at[38,'team']='49ers'

cities_nfl = cities.iloc[:,[0,1,2]]
cities_nfl.replace({'NFL':r'\[.+\]'}, {'NFL':""},regex = True,inplace = True)

cities_nfl.at[0,'NFL'] = 'Giants'
nr = pd.Series({cities_nfl.columns[0]:cities_nfl.iat[0,0],cities_nfl.columns[1]:cities_nfl.iat[0,1],'NFL':'Jets'})
cities_nfl = cities_nfl.append(nr,ignore_index = True)

cities_nfl.at[1,'NFL'] = 'Rams'
nr = pd.Series({cities_nfl.columns[0]:cities_nfl.iat[1,0],cities_nfl.columns[1]:cities_nfl.iat[1,1],'NFL':'Chargers'})
cities_nfl = cities_nfl.append(nr,ignore_index = True)

cities_nfl.at[2,'NFL'] = '49ers'
nr = pd.Series({cities_nfl.columns[0]:cities_nfl.iat[2,0],cities_nfl.columns[1]:cities_nfl.iat[2,1],'NFL':'Raiders'})
cities_nfl = cities_nfl.append(nr,ignore_index = True)

new_df_nfl = cities_nfl.merge(nfl_df,left_on='NFL',right_on='team',how='right')
new_df_nfl['W'] = pd.to_numeric(new_df_nfl['W'])
new_df_nfl['L'] = pd.to_numeric(new_df_nfl['L'])
new_df_nfl['Population'] = pd.to_numeric(new_df_nfl['Population'])


def nfl_correlation(): 
    
    ans = corr_data(new_df_nfl)
    population_by_region = ans['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = ans['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    return stats.pearsonr(population_by_region, win_loss_by_region)

print(nfl_correlation())

<class 'pandas.core.series.Series'>
(0.004922112149349393, 0.979783345836372)


## Question 5
Testing the hypothesis is that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. 

In [47]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def sports_team_performance():
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    print(p_values)
    corrDict = {'NFL':new_df_nfl,'NBA':new_df_nba,'NHL':new_df_nhl,'MLB':new_df_mlb}
    
    for s1 in sports:
        for s2 in sports:
            ps1 = pd.DataFrame(corr_data(corrDict[s1])['ratio'])
            ps2 = pd.DataFrame(corr_data(corrDict[s2])['ratio'])
            
            ps = ps1.merge(ps2,on='Metropolitan area',how='inner')
            stat,p=stats.ttest_rel(ps.iloc[:,0],ps.iloc[:,1])
            p_values.loc[s1,s2] = p
            

print(sports_team_performance())

     NFL  NBA  NHL  MLB
NFL  NaN  NaN  NaN  NaN
NBA  NaN  NaN  NaN  NaN
NHL  NaN  NaN  NaN  NaN
MLB  NaN  NaN  NaN  NaN
          NFL       NBA       NHL       MLB
NFL       NaN  0.941792  0.030959  0.802384
NBA  0.941792       NaN  0.022316  0.951046
NHL  0.030959  0.022316       NaN  0.000712
MLB  0.802384  0.951046  0.000712       NaN
